<font size="6">**Movie Data Preprocessing**</font>

<font size="5">**Part 1:** Webscraping</font>

In this part, I focused on extracting valuable data from the "IMDb Top 250" webpage. To achieve this, I first used the requests library to download the webpage's HTML content. Then, I parsed the HTML content using the BeautifulSoup library, which enabled me to extract the necessary information from the webpage's HTML tags. I extracted the movie titles, years of release, and IMDb ratings for each movie by selecting the appropriate HTML tags using the select() method.


In [214]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import math
 
 
# Downloading imdb top 250 movie's data
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
movies = soup.select('td.titleColumn')
ratings = [b.attrs.get('data-value')
        for b in soup.select('td.posterColumn span[name=ir]')]
 
# create an empty dataframe for storing
# movie information
imdbdf = pd.DataFrame()
 
# Iterating over movies to extract
# each movie's details
for index in range(0, len(movies)):
     
    # Separating movie into: 'place',
    # 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]
    data = {"place": place,
            "movie_title": movie_title,
            "imdb_score": ratings[index],
            "year": year,
            }
    imdbdf = imdbdf.append(data, ignore_index=True)
 


<font size="5">**Part 2:** Rotten Tomatoes Dataset</font>

In this analysis, the dataset used was obtained from Kaggle, which contains information on top-rated movies from Rotten Tomatoes: https://www.kaggle.com/datasets/thedevastator/rotten-tomatoes-top-movies-ratings-and-technical. To prepare the dataset for analysis, irrelevant variables were dropped. The variables that were deemed relevant for analysis, such as the rating, genre, runtime, and box office variables, were reformatted to ensure they are better suited for the regression analysis.

To further enhance the analysis, a sentiment score was calculated for each movie synopsis. This was done using a lexicon-based approach with the VADER tool. This tool assigns a sentiment score to text based on a pre-built lexicon that contains positive and negative words. The sentiment score ranges from -1 (most negative) to 1 (most positive) and 0 represents a neutral score.

In [215]:
# Read the CSV file into a pandas dataframe
df = pd.read_csv('tomatoes.csv')

# Drop the specified columns
columns_to_drop = ['consensus', 'total_ratings', 'original_language', 
                   'release_date_(streaming)', 'production_co', 
                   'sound_mix', 'aspect_ratio', 'link', 'producer',
                  'release_date_(theaters)',]
df = df.drop(columns=columns_to_drop)


#Remove rating explanation from rating column
df['rating'] = df['rating'].str.replace(r'\(.*\)', '')




In [216]:
# Filter out rows with missing or null values in the "genre" column
genrelist = df.loc[~pd.isna(df['genre']), 'genre'].tolist()


# Split each string into a list of words, and combine all the lists
all_words = [word for sublist in [string.split(',') for string in genrelist] for word in sublist]

all_words = [s.strip() for s in all_words]

# Get the unique words from the combined list
unique_words = list(set(all_words))

# Print the resulting list of unique words
print(unique_words)

['history', 'fantasy', 'other', 'comedy', 'sports and fitness', 'western', 'action', 'kids and family', 'anime', 'biography', 'adventure', 'animation', 'horror', 'music', 'crime', 'sci fi', 'romance', 'musical', 'documentary', 'mystery and thriller', 'gay and lesbian', 'war', 'drama']


In [217]:

# create a new dataframe with dummies
dummies_df = pd.get_dummies(df['genre'].apply(pd.Series).stack()).sum(level=0)

# reindex the new dataframe with the unique genres
dummies_df = dummies_df.reindex(columns=unique_words, fill_value=0)

# concatenate the original dataframe with the new dummies dataframe
df = pd.concat([df, dummies_df], axis=1)


In [218]:
df['rating'] = df['rating'].str.strip()
# create a new dataframe with dummies
ratingdummies = pd.get_dummies(df['rating'])

# concatenate the original dataframe with the new dummies dataframe
df = pd.concat([df, ratingdummies], axis=1)




In [219]:
#Converting runtime to minutes

# Define a function to convert the 'runtime' values to total minutes

def convert_to_minutes(runtime):
    if isinstance(runtime, str):
        if 'h ' in runtime:
            hours, minutes = runtime.split('h ')
            return int(hours) * 60 + int(minutes[:-1])
        else:
            return int(runtime[:-1])
    else:
        return math.nan



# Apply the function to the 'runtime' column to create a new 'total_minutes' column
df['runtime'] = df['runtime'].apply(convert_to_minutes)


In [220]:
# Define a function to convert the 'box_office_(gross_usa)' values to numbers
def convert_to_numbers(box_office):
    if isinstance(box_office, str):
        if box_office[-1] == 'M':
            return float(box_office[1:-1]) * 1000000
        elif box_office[-1] == 'K':
            return float(box_office[1:-1]) * 1000
    return math.nan


# Apply the function to the 'box_office_(gross_usa)' column to create a new 'boxoffice' column
df['boxoffice'] = df['box_office_(gross_usa)'].apply(convert_to_numbers)

# Convert the 'boxoffice' column to float
df['boxoffice'] = pd.to_numeric(df['boxoffice'])

# Drop the original 'box_office_(gross_usa)' column
df.drop('box_office_(gross_usa)', axis=1, inplace=True)



In [221]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Drop rows with NaN or float in 'synopsis' column
df = df.dropna(subset=['synopsis'])
df = df[~df['synopsis'].apply(lambda x: isinstance(x, float))]

# Instantiate the analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to calculate the sentiment score for each synopsis
def get_sentiment_score(text):
    score = analyzer.polarity_scores(text)
    return score['compound']

# Apply the function to the 'synopsis' column to create a new 'sentiment_score' column
df['sentiment_score'] = df['synopsis'].apply(get_sentiment_score)


<font size="5">**Part 3:** Export Data</font>

Exporting data to a .csv file to analyze in SQL and Tableau!

In [223]:
# Export imdbdf to a CSV file
imdbdf.to_csv('processed_imdb.csv', index=False)

# Export df to a CSV file
df.to_csv('processed_tomato.csv', index=False)
